In [1]:
import csv
import math
import json
from typing import List, Dict, Optional, Tuple, Any
from collections import defaultdict

In [ ]:
class EmbeddingStorage:
    """
    Estructura de datos para almacenar embeddings de usuarios organizados en chunks
    para lectura rápida basada en similitud coseno
    """
    
    def __init__(self, cosine_threshold: float = 0.8, chunk_size_limit: int = 1000):
        """
        Inicializa el almacenamiento de embeddings
        
        Args:
            cosine_threshold: Umbral de similitud coseno para agrupar embeddings
            chunk_size_limit: Límite máximo de embeddings por chunk
        """
        self.cosine_threshold = cosine_threshold
        self.chunk_size_limit = chunk_size_limit
        self.chunks = []  # Lista de chunks
        self.user_index = {}  # Mapeo usuario -> (chunk_id, posición_en_chunk)
        self.movie_names = []  # Lista de nombres de películas
        self.user_names = []  # Lista de nombres de usuarios
        self.stats = {
            'total_users': 0,
            'total_chunks': 0,
            'avg_chunk_size': 0,
            'embedding_dimension': 0
        }
    
    def load_csv(self, csv_path: str) -> None:
        """
        Carga datos desde un archivo CSV
        
        Args:
            csv_path: Ruta al archivo CSV
        """
        print(f"Cargando datos desde: {csv_path}")
        
        with open(csv_path, 'r', encoding='utf-8') as file:
            reader = csv.reader(file)
            
            # Leer encabezados (usuarios)
            header = next(reader)
            self.user_names = header[1:]  # Omitir primera columna (películas)
            
            # Leer datos
            rating_matrix = []
            for row in reader:
                movie_name = row[0]
                self.movie_names.append(movie_name)
                
                # Convertir ratings a float, manejar valores faltantes
                ratings = []
                for rating_str in row[1:]:
                    try:
                        rating = float(rating_str) if rating_str.strip() else 0.0
                        ratings.append(rating)
                    except ValueError:
                        ratings.append(0.0)
                
                rating_matrix.append(ratings)
        
        print(f"Cargados: {len(self.movie_names)} películas, {len(self.user_names)} usuarios")
        
        # Procesar embeddings
        self._process_embeddings(rating_matrix)
    
    def _process_embeddings(self, rating_matrix: List[List[float]]) -> None:
        """
        Procesa la matriz de calificaciones y crea embeddings organizados en chunks
        
        Args:
            rating_matrix: Matriz con calificaciones [película][usuario]
        """
        print("Generando embeddings...")
        
        num_users = len(self.user_names)
        num_movies = len(self.movie_names)
        
        # Transponer matriz para obtener [usuario][película]
        user_embeddings = []
        for user_idx in range(num_users):
            embedding = []
            for movie_idx in range(num_movies):
                embedding.append(rating_matrix[movie_idx][user_idx])
            user_embeddings.append(embedding)
        
        # Normalizar embeddings
        normalized_embeddings = []
        for embedding in user_embeddings:
            normalized = self._normalize_embedding(embedding)
            normalized_embeddings.append(normalized)
        
        # Crear chunks secuencialmente
        self._create_chunks(normalized_embeddings)
        
        # Actualizar estadísticas
        self._update_stats()
        
        print(f"Embeddings organizados en {len(self.chunks)} chunks")
    
    def _normalize_embedding(self, embedding: List[float]) -> List[float]:
        """
        Normaliza un embedding usando z-score
        
        Args:
            embedding: Vector de calificaciones
            
        Returns:
            Embedding normalizado
        """
        # Calcular media y desviación estándar
        mean_val = sum(embedding) / len(embedding)
        variance = sum((x - mean_val) ** 2 for x in embedding) / len(embedding)
        std_dev = math.sqrt(variance) if variance > 0 else 1.0
        
        # Normalizar
        normalized = [(x - mean_val) / std_dev for x in embedding]
        return normalized
    
    def _cosine_similarity(self, vec1: List[float], vec2: List[float]) -> float:
        """
        Calcula similitud coseno entre dos vectores
        
        Args:
            vec1, vec2: Vectores a comparar
            
        Returns:
            Similitud coseno [-1, 1]
        """
        # Producto punto
        dot_product = sum(a * b for a, b in zip(vec1, vec2))
        
        # Normas
        norm1 = math.sqrt(sum(a * a for a in vec1))
        norm2 = math.sqrt(sum(b * b for b in vec2))
        
        # Evitar división por cero
        if norm1 == 0 or norm2 == 0:
            return 0.0
        
        return dot_product / (norm1 * norm2)
    
    def _create_chunks(self, embeddings: List[List[float]]) -> None:
        """
        Crea chunks organizando embeddings por similitud secuencial
        
        Args:
            embeddings: Lista de embeddings normalizados
        """
        if not embeddings:
            return
        
        # Inicializar primer chunk
        current_chunk = {
            'id': 0,
            'embeddings': [embeddings[0]],
            'users': [self.user_names[0]],
            'centroid': embeddings[0].copy(),
            'size': 1
        }
        
        # Mapear usuario al chunk
        self.user_index[self.user_names[0]] = (0, 0)
        
        # Procesar resto de embeddings secuencialmente
        for i in range(1, len(embeddings)):
            current_embedding = embeddings[i]
            current_user = self.user_names[i]
            
            # Calcular similitud con centroide del chunk actual
            similarity = self._cosine_similarity(current_embedding, current_chunk['centroid'])
            
            # Verificar si debe ir al chunk actual
            if (similarity >= self.cosine_threshold and 
                current_chunk['size'] < self.chunk_size_limit):
                
                # Agregar al chunk existente
                current_chunk['embeddings'].append(current_embedding)
                current_chunk['users'].append(current_user)
                current_chunk['size'] += 1
                
                # Actualizar centroide
                self._update_centroid(current_chunk)
                
                # Mapear usuario
                self.user_index[current_user] = (current_chunk['id'], current_chunk['size'] - 1)
                
            else:
                # Finalizar chunk actual
                self.chunks.append(current_chunk)
                
                # Crear nuevo chunk
                new_chunk_id = len(self.chunks)
                current_chunk = {
                    'id': new_chunk_id,
                    'embeddings': [current_embedding],
                    'users': [current_user],
                    'centroid': current_embedding.copy(),
                    'size': 1
                }
                
                # Mapear usuario
                self.user_index[current_user] = (new_chunk_id, 0)
        
        # Agregar último chunk
        self.chunks.append(current_chunk)
    
    def _update_centroid(self, chunk: Dict) -> None:
        """
        Actualiza el centroide de un chunk
        
        Args:
            chunk: Diccionario del chunk
        """
        if not chunk['embeddings']:
            return
        
        # Calcular nuevo centroide
        embedding_dim = len(chunk['embeddings'][0])
        new_centroid = [0.0] * embedding_dim
        
        for embedding in chunk['embeddings']:
            for i in range(embedding_dim):
                new_centroid[i] += embedding[i]
        
        # Promediar
        for i in range(embedding_dim):
            new_centroid[i] /= len(chunk['embeddings'])
        
        chunk['centroid'] = new_centroid
    
    def _update_stats(self) -> None:
        """
        Actualiza estadísticas del sistema
        """
        self.stats['total_users'] = len(self.user_names)
        self.stats['total_chunks'] = len(self.chunks)
        self.stats['embedding_dimension'] = len(self.movie_names)
        
        if self.chunks:
            total_size = sum(chunk['size'] for chunk in self.chunks)
            self.stats['avg_chunk_size'] = total_size / len(self.chunks)
    
    def get_user_embedding(self, user_name: str) -> Optional[List[float]]:
        """
        Obtiene el embedding de un usuario específico
        
        Args:
            user_name: Nombre del usuario
            
        Returns:
            Embedding del usuario o None si no existe
        """
        if user_name not in self.user_index:
            return None
        
        chunk_id, position = self.user_index[user_name]
        return self.chunks[chunk_id]['embeddings'][position]
    
    def get_similar_users(self, user_name: str, max_results: int = 10) -> List[Tuple[str, float]]:
        """
        Encuentra usuarios similares a uno dado
        
        Args:
            user_name: Nombre del usuario
            max_results: Número máximo de resultados
            
        Returns:
            Lista de tuplas (usuario, similitud) ordenadas por similitud
        """
        if user_name not in self.user_index:
            return []
        
        target_embedding = self.get_user_embedding(user_name)
        chunk_id, _ = self.user_index[user_name]
        
        # Buscar en el mismo chunk primero
        similar_users = []
        target_chunk = self.chunks[chunk_id]
        
        for i, (user, embedding) in enumerate(zip(target_chunk['users'], target_chunk['embeddings'])):
            if user != user_name:
                similarity = self._cosine_similarity(target_embedding, embedding)
                similar_users.append((user, similarity))
        
        # Buscar en chunks adyacentes si es necesario
        if len(similar_users) < max_results:
            adjacent_chunks = []
            if chunk_id > 0:
                adjacent_chunks.append(self.chunks[chunk_id - 1])
            if chunk_id < len(self.chunks) - 1:
                adjacent_chunks.append(self.chunks[chunk_id + 1])
            
            for chunk in adjacent_chunks:
                for user, embedding in zip(chunk['users'], chunk['embeddings']):
                    similarity = self._cosine_similarity(target_embedding, embedding)
                    similar_users.append((user, similarity))
        
        # Ordenar por similitud y limitar resultados
        similar_users.sort(key=lambda x: x[1], reverse=True)
        return similar_users[:max_results]
    
    def get_chunk_info(self) -> List[Dict]:
        """
        Obtiene información de todos los chunks
        
        Returns:
            Lista de diccionarios con información de cada chunk
        """
        chunk_info = []
        for chunk in self.chunks:
            # Calcular similitud promedio dentro del chunk
            avg_similarity = 0.0
            comparisons = 0
            
            for i in range(len(chunk['embeddings'])):
                for j in range(i + 1, len(chunk['embeddings'])):
                    similarity = self._cosine_similarity(
                        chunk['embeddings'][i], 
                        chunk['embeddings'][j]
                    )
                    avg_similarity += similarity
                    comparisons += 1
            
            if comparisons > 0:
                avg_similarity /= comparisons
            
            chunk_info.append({
                'chunk_id': chunk['id'],
                'size': chunk['size'],
                'avg_similarity': avg_similarity,
                'users_preview': chunk['users'][:3]  # Primeros 3 usuarios
            })
        
        return chunk_info
    
    def search_by_preferences(self, movie_preferences: Dict[str, float], 
                            max_results: int = 10) -> List[Tuple[str, float]]:
        """
        Busca usuarios con preferencias similares
        
        Args:
            movie_preferences: Diccionario {película: calificación}
            max_results: Número máximo de resultados
            
        Returns:
            Lista de tuplas (usuario, similitud)
        """
        # Crear embedding de preferencias
        preference_embedding = []
        for movie in self.movie_names:
            rating = movie_preferences.get(movie, 0.0)
            preference_embedding.append(rating)
        
        # Normalizar
        preference_embedding = self._normalize_embedding(preference_embedding)
        
        # Buscar usuarios similares
        similar_users = []
        for chunk in self.chunks:
            for user, embedding in zip(chunk['users'], chunk['embeddings']):
                similarity = self._cosine_similarity(preference_embedding, embedding)
                similar_users.append((user, similarity))
        
        # Ordenar y limitar
        similar_users.sort(key=lambda x: x[1], reverse=True)
        return similar_users[:max_results]
    
    def save_structure(self, filepath: str) -> None:
        """
        Guarda la estructura en un archivo JSON
        
        Args:
            filepath: Ruta del archivo de salida
        """
        data = {
            'cosine_threshold': self.cosine_threshold,
            'chunk_size_limit': self.chunk_size_limit,
            'chunks': self.chunks,
            'user_index': self.user_index,
            'movie_names': self.movie_names,
            'user_names': self.user_names,
            'stats': self.stats
        }
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        
        print(f"Estructura guardada en: {filepath}")
    
    def load_structure(self, filepath: str) -> None:
        """
        Carga la estructura desde un archivo JSON
        
        Args:
            filepath: Ruta del archivo a cargar
        """
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        self.cosine_threshold = data['cosine_threshold']
        self.chunk_size_limit = data['chunk_size_limit']
        self.chunks = data['chunks']
        self.user_index = data['user_index']
        self.movie_names = data['movie_names']
        self.user_names = data['user_names']
        self.stats = data['stats']
        
        print(f"Estructura cargada desde: {filepath}")
    
    def print_stats(self) -> None:
        """
        Imprime estadísticas del sistema
        """
        print("\n=== ESTADÍSTICAS DEL SISTEMA ===")
        print(f"Total de usuarios: {self.stats['total_users']}")
        print(f"Total de chunks: {self.stats['total_chunks']}")
        print(f"Tamaño promedio de chunk: {self.stats['avg_chunk_size']:.2f}")
        print(f"Dimensión de embeddings: {self.stats['embedding_dimension']}")
        print(f"Umbral de similitud: {self.cosine_threshold}")
        print(f"Límite de chunk: {self.chunk_size_limit}")
        
        # Información de chunks
        print("\n=== INFORMACIÓN DE CHUNKS ===")
        for i, chunk in enumerate(self.chunks):
            print(f"Chunk {i}: {chunk['size']} usuarios")
            if chunk['size'] <= 5:
                print(f"  Usuarios: {', '.join(chunk['users'])}")
            else:
                print(f"  Usuarios: {', '.join(chunk['users'][:3])}... (+{chunk['size']-3} más)")


In [5]:
def demo():
    """
    Demostración del sistema
    """
    print("=== DEMOSTRACIÓN DEL SISTEMA DE EMBEDDINGS ===\n")
    
    
    # Inicializar sistema
    storage = EmbeddingStorage(cosine_threshold=0.7, chunk_size_limit=100)
    
    # Cargar datos
    storage.load_csv("Movie_Ratings.csv")
    
    # Mostrar estadísticas
    storage.print_stats()
    
    # Buscar usuarios similares
    print("\n=== BÚSQUEDA DE USUARIOS SIMILARES ===")
    similar = storage.get_similar_users("Usuario_1", max_results=5)
    print(f"Usuarios similares a Usuario_1:")
    for user, similarity in similar:
        print(f"  {user}: {similarity:.3f}")
    
    # Búsqueda por preferencias
    print("\n=== BÚSQUEDA POR PREFERENCIAS ===")
    preferences = {
        "Película_1": 5.0,
        "Película_2": 4.5,
        "Película_3": 4.0
    }
    
    matches = storage.search_by_preferences(preferences, max_results=5)
    print("Usuarios con preferencias similares:")
    for user, similarity in matches:
        print(f"  {user}: {similarity:.3f}")
    
    # Información de chunks
    print("\n=== INFORMACIÓN DETALLADA DE CHUNKS ===")
    chunk_info = storage.get_chunk_info()
    for info in chunk_info:
        print(f"Chunk {info['chunk_id']}: {info['size']} usuarios, "
              f"similitud promedio: {info['avg_similarity']:.3f}")
    
    # Guardar estructura
    storage.save_structure("embedding_structure.json")
    
    return storage


In [6]:

storage = demo()

=== DEMOSTRACIÓN DEL SISTEMA DE EMBEDDINGS ===

Cargando datos desde: Movie_Ratings.csv
Cargados: 25 películas, 25 usuarios
Generando embeddings...
Embeddings organizados en 25 chunks

=== ESTADÍSTICAS DEL SISTEMA ===
Total de usuarios: 25
Total de chunks: 25
Tamaño promedio de chunk: 1.00
Dimensión de embeddings: 25
Umbral de similitud: 0.7
Límite de chunk: 100

=== INFORMACIÓN DE CHUNKS ===
Chunk 0: 1 usuarios
  Usuarios: Patrick C
Chunk 1: 1 usuarios
  Usuarios: Heather
Chunk 2: 1 usuarios
  Usuarios: Bryan
Chunk 3: 1 usuarios
  Usuarios: Patrick T
Chunk 4: 1 usuarios
  Usuarios: Thomas
Chunk 5: 1 usuarios
  Usuarios: aaron
Chunk 6: 1 usuarios
  Usuarios: vanessa
Chunk 7: 1 usuarios
  Usuarios: greg
Chunk 8: 1 usuarios
  Usuarios: brian
Chunk 9: 1 usuarios
  Usuarios: ben
Chunk 10: 1 usuarios
  Usuarios: Katherine
Chunk 11: 1 usuarios
  Usuarios: Jonathan
Chunk 12: 1 usuarios
  Usuarios: Zwe
Chunk 13: 1 usuarios
  Usuarios: Erin
Chunk 14: 1 usuarios
  Usuarios: Chris
Chunk 15: 1 usu